In [1]:
# Install necessary libraries
!pip install albumentations segmentation-models-pytorch torchmetrics --quiet
!pip install --upgrade albumentations[imgaug] --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.9/273.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 7.6 MB/s eta 0:00:00


In [2]:
# Import libraries
import os
import zipfile
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, random_split
import cv2
import numpy as np
import torchmetrics
import segmentation_models_pytorch as smp

In [3]:
# Download and extract dataset
DATA_URL = "https://github.com/ongchinkiat/LyftPerceptionChallenge/releases/download/v0.1/carla-capture-20180513A.zip"
DATA_PATH = "/content/data"
ZIP_PATH = "/content/data.zip"

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
    !wget -q $DATA_URL -O $ZIP_PATH
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(DATA_PATH)
    print("Dataset downloaded and extracted!")

# Dataset path setup
IMAGE_PATH = os.path.join(DATA_PATH, "CameraRGB")
MASK_PATH = os.path.join(DATA_PATH, "CameraSeg")

# Custom dataset class
class CityScapes(torch.utils.data.Dataset):
    def __init__(self, image_folder_path, mask_folder_path, transforms=None):
        self.images_folder = image_folder_path
        self.masks_folder = mask_folder_path
        self.images = os.listdir(self.images_folder)
        self.transforms = transforms

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        file_name = self.images[index]
        image = cv2.imread(os.path.join(self.images_folder, file_name))
        mask = cv2.imread(os.path.join(self.masks_folder, file_name))

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = mask[:, :, 2]  # Use the blue channel as segmentation mask

        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.unsqueeze(0)

# Data transformations
transforms = A.Compose([
    A.Resize(256, 256),
    A.Normalize(),
    ToTensorV2()
])

# Load dataset
dataset = CityScapes(IMAGE_PATH, MASK_PATH, transforms=transforms)

# Train-test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

Dataset downloaded and extracted!


In [4]:
# U-Net model implementation
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(feature * 2, feature, kernel_size=2, stride=2)
            )
            self.ups.append(DoubleConv(feature * 2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]
            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)

# Training and evaluation functions
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    dice_score = torchmetrics.Dice(num_classes=13, average='micro').to(device)
    with torch.no_grad():
        for data, target in dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target.long())
            total_loss += loss.item()
            dice_score.update(output, target.long())
    return total_loss / len(dataloader), dice_score.compute()

# Training pipeline
def train(model, train_dataset, test_dataset, num_epochs=20, batch_size=8):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    criterion = smp.losses.DiceLoss(mode='multiclass')
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    best_loss = float('inf')

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        test_loss, dice_score = evaluate(model, test_loader, criterion, device)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Dice Score: {dice_score:.4f}")

        if test_loss < best_loss:
            best_loss = test_loss
            torch.save(model.state_dict(), "best_model.pth")
            print("Model saved!")

# Initialize and train the model
model = UNET(in_channels=3, out_channels=13)  # Assuming 13 classes for segmentation
train(model, train_dataset, test_dataset)

Epoch 1/20, Train Loss: 0.6767, Test Loss: 0.5410, Dice Score: 0.8915
Model saved!
Epoch 2/20, Train Loss: 0.4783, Test Loss: 0.4782, Dice Score: 0.8654
Model saved!
Epoch 3/20, Train Loss: 0.3943, Test Loss: 0.3877, Dice Score: 0.8832
Model saved!
Epoch 4/20, Train Loss: 0.3428, Test Loss: 0.3301, Dice Score: 0.9240
Model saved!
Epoch 5/20, Train Loss: 0.3146, Test Loss: 0.4500, Dice Score: 0.7743
Epoch 6/20, Train Loss: 0.2974, Test Loss: 0.2888, Dice Score: 0.9377
Model saved!
Epoch 7/20, Train Loss: 0.2745, Test Loss: 0.2700, Dice Score: 0.9416
Model saved!
Epoch 8/20, Train Loss: 0.2670, Test Loss: 0.2799, Dice Score: 0.9370
Epoch 9/20, Train Loss: 0.2532, Test Loss: 0.2730, Dice Score: 0.9358
Epoch 10/20, Train Loss: 0.2334, Test Loss: 0.2559, Dice Score: 0.9305
Model saved!
Epoch 11/20, Train Loss: 0.2256, Test Loss: 0.2212, Dice Score: 0.9422
Model saved!
Epoch 12/20, Train Loss: 0.2183, Test Loss: 0.2331, Dice Score: 0.9385
Epoch 13/20, Train Loss: 0.2046, Test Loss: 0.1960, D